In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford,StabilizerState #clifford class, python based simulator
from qiskit.providers.aer import AerSimulator #AerSimulator C++ based simulator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Prepare a GHZ circuit
num_qubits = 3

qc = QuantumCircuit(num_qubits)
qc.h(0)
for i in range(1,num_qubits):
    qc.cx(0,i)
    
# Initialise the Clifford class
cliff = Clifford(qc)

# Print the stabilizer generators
print(cliff.stabilizer)

In [ ]:
# reconstruct circuit from clifford (works for any clifford)
print(cliff.to_circuit())

In [ ]:
# number of shots
shots = 100

# Get the sample counts using StabilizerState (python-based simulator)
counts = StabilizerState(qc).sample_counts(shots)
print ("GHZ state outcome counts:", counts)

# Get the probabilities using StabilizerState (python-based simulator) 
prob = StabilizerState(qc).probabilities_dict(decimals=2)
print ("GHZ state outcome probabilities:", prob)

In [ ]:
# Add n measurement gates to the GHZ circuit
qcm = qc.copy()
qcm.measure_all()

In [ ]:
# Prepare the simulator (method=stabilizer is optional,
# for clifford circuits it will be used automatically if no other method is provided
simulator = AerSimulator(method='stabilizer')

shots=1000
# Run the C++ based simulator Simulator on GHZ circuit
result = simulator.run(qcm, shots=shots).result()
result.get_counts(0) # Get the counts

In [ ]:
H = np.matrix([[(1/np.sqrt(2)), (1/np.sqrt(2))], [(1/np.sqrt(2)), (-1/np.sqrt(2))]])
Sa = np.matrix(([[1, 0], [0, -1j]]))
I = np.matrix([[1, 0], [0, 1]])

Us = [I,H,H@Sa]
def fidelity(qc,num_qubits):   
    fidelities = []
    fid_avg = []
    t = 0
    while True:
        t+=1
        qcm = qc.copy()
        m_int = np.random.randint(3,size=num_qubits)
        # print(m_int)
        
        for i,k in enumerate(m_int):
            if k == 1:
                qcm.h(i)
            if k == 2:
                qcm.sdg(i)
                qcm.h(i)
        qcm.measure_all()
        result = simulator.run(qcm, shots=1).result()
        for r in result.get_counts(0): 
            r = r[::-1]
            p = I
            for i in range(len(r)):
                b = np.array([1,0])
                # print(r[i])
                U = Us[m_int[i]]
                # print(U)
                if r[i] == '1':
                    
                    b = np.matrix([0,1])
                
                if i == 0:
                    p = np.outer(3*U.H@b.T,b@U)-I
                else:
                    pi = np.outer(3*U.H@b.T,b@U)-I
                    p = np.kron(p,pi)
        p_00 = p[0,0]
        p_01 = p[0,2**num_qubits-1]
        p_10 = p[2**num_qubits-1,0]
        p_11 = p[2**num_qubits-1,2**num_qubits-1]
        fid = (p_00 +p_01+p_10+p_11)/2
        fidelities.append(fid)
        fid_avg.append(np.mean(fidelities))
        
        if all(0.99 <= val <= 1 for val in fid_avg[-5:]):
            break 
        if t>10000:
            break
            
    # plt.plot(range(1, t+1), fid_avg)  # Assuming t starts from 1
    # plt.xlabel('Iterations (t)')
    # plt.ylabel('Average Fidelity (fid_avg)')
    # plt.title('Average Fidelity over Iterations')
    # plt.show()
        # if t % 10 == 0:
        #     print(f" {t} shadows -> Fidelity: {np.mean(fidelities)}")
        
    return (t+1, np.mean(fidelities))
required = {}
num_qubits_list = []
shadows_list = []
for i in range(2,11):
    # Prepare a GHZ circuit
    num_qubits = i
    
    qc = QuantumCircuit(num_qubits)
    qc.h(0)
    for n in range(1,num_qubits):
        qc.cx(0,n)

    # Initialise the Clifford class
    cliff = Clifford(qc)

    
    required[i] = np.mean([fidelity(qc,num_qubits)[0] for _ in range(3)])
    
    num_qubits_list.append(i)
    shadows_list.append(required[i])  
    print(f"For {i} qubits - The number of shadows necessary: {required[i]}")

plt.plot(num_qubits_list, shadows_list, marker='o')
plt.xlabel('Qubits')
plt.ylabel('Number of Shadows Necessary')
plt.title('Number of Qubits vs Number of Shadows Necessary')
plt.grid(True)
plt.show()
    